In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv("202107-citibike-tripdata.csv")

In [ ]:
df.head()

start_station_counts = df.groupby(by=["start_station_name"]).size().sort_values(ascending=False)
end_station_counts = df.groupby(by=["end_station_name"]).size().sort_values(ascending=False)

In [ ]:
start_station_counts.head()

In [ ]:
start_station_counts.head()

In [ ]:
stations_df=durations_df[(durations_df['start_station_name']=="7 Ave & Central Park South") | 
                        (durations_df['end_station_name']=="7 Ave & Central Park South") |
                        (durations_df['start_station_name']=="E 17 St & Broadway") | 
                        (durations_df['end_station_name']=="E 17 St & Broadway") | 
                        (durations_df['start_station_name']=="W 21 St & 6 Ave") | 
                        (durations_df['end_station_name']=="W 21 St & 6 Ave")]

In [ ]:
starts = []
ends = []
for index, row in stations_df.iterrows():
    start_datetime_object = datetime.strptime((row["started_at"]), "%Y-%m-%d %H:%M:%S")
    end_datetime_object = datetime.strptime((row["ended_at"]), "%Y-%m-%d %H:%M:%S")
    starts.append(start_datetime_object)
    ends.append(end_datetime_object)
    
    if len(starts) % 10000 == 0:
        print(len(starts))

In [ ]:
start_datetime = []
for date in starts:
    if date.weekday() != 5 and date.weekday() != 6:
        start_datetime.append(date)    

In [ ]:
end_datetime = []
for date in ends:
    if date.weekday() != 5 and date.weekday() != 6:
        end_datetime.append(date)   

In [ ]:
stations_weekdays = pd.DataFrame(columns = stations_df.columns)

for index, row in stations_df.iterrows():
    if (datetime.strptime((row["started_at"]), "%Y-%m-%d %H:%M:%S").weekday() != 5
    and datetime.strptime((row["started_at"]), "%Y-%m-%d %H:%M:%S").weekday() != 6
    and datetime.strptime((row["ended_at"]), "%Y-%m-%d %H:%M:%S").weekday() != 5
    and datetime.strptime((row["ended_at"]), "%Y-%m-%d %H:%M:%S").weekday() != 6):
        stations_weekdays.loc[index] = stations_df.loc[index]
    
    if len(stations_weekdays) % 10000 == 0:
        print(len(stations_weekdays))

In [ ]:
stations_weekdays = stations_weekdays.loc[:, stations_weekdays.columns != 'index']

In [ ]:
stations_weekdays = stations_weekdays.loc[:, stations_weekdays.columns != 'level_0']

In [ ]:
stations_weekdays["start"] = starts
stations_weekdays["end"] = ends

In [ ]:
stations_weekdays = stations_weekdays.loc[:, stations_weekdays.columns != 'started_at']
stations_weekdays = stations_weekdays.loc[:, stations_weekdays.columns != 'ended_at']
stations_weekdays["hour_start"] = [time.hour for time in stations_weekdays["start"]]

In [ ]:
day_block = []
for index, row in stations_weekdays.iterrows():
    if row["hour_start"] < 2:
        day_block.append(1)
    elif row["hour_start"] < 14:
        day_block.append(0)
    else:
        day_block.append(1)

In [ ]:
#Step 1 -- turn "started_at" and "ended_at" columns to datetime objects
starts = []
ends = []
for index, row in stations_weekdays.iterrows():
    start_datetime_object = datetime.strptime((row["started_at"]), "%Y-%m-%d %H:%M:%S")
    end_datetime_object = datetime.strptime((row["ended_at"]), "%Y-%m-%d %H:%M:%S")
    starts.append(start_datetime_object)
    ends.append(end_datetime_object)
    
    if len(starts) % 10000 == 0:
        print(len(starts))

In [ ]:
stations_weekdays["day_block"] = day_block

In [ ]:
#discretize blocks into periods of 5 or 10 minutes
stations_weekdays["time"] = [time.time() for time in stations_weekdays["start"]]

In [ ]:
time_block = [math.floor((time.hour * 60 + time.minute) / 5) for time in stations_weekdays["time"]]

In [ ]:
stations_weekdays["time_block"] = time_block

In [ ]:
stations_weekdays = stations_weekdays.loc[:, stations_weekdays.columns != 'hour']
stations_weekdays = stations_weekdays.loc[:, stations_weekdays.columns != 'hour_start']
stations_weekdays = stations_weekdays.loc[:, stations_weekdays.columns != 'hour_min']

In [ ]:
day_of_month = []
for index, row in stations_weekdays.iterrows():
    day_of_month.append(row["start"].day)
stations_weekdays["day_of_month"] = day_of_month

In [ ]:
#station_capacities -- collect from google?
morning_data = stations_weekdays[stations_weekdays["day_block"] == 0]
evening_data = stations_weekdays[stations_weekdays["day_block"] == 1]

In [ ]:
morning_data_station1=morning_data[(morning_data['start_station_name']=="7 Ave & Central Park South") | 
                        (durations_df['end_station_name']=="7 Ave & Central Park South")]
evening_data_station1=evening_data[(evening_data['start_station_name']=="7 Ave & Central Park South") | 
                        (durations_df['end_station_name']=="7 Ave & Central Park South")]
morning_data_station2=morning_data[(morning_data['start_station_name']=="E 17 St & Broadway") | 
                        (durations_df['end_station_name']=="E 17 St & Broadway")]
evening_data_station2=evening_data[(evening_data['start_station_name']=="E 17 St & Broadway") | 
                        (durations_df['end_station_name']=="E 17 St & Broadway")]
morning_data_station3=morning_data[(morning_data['start_station_name']=="W 21 St & 6 Ave") | 
                        (durations_df['end_station_name']=="W 21 St & 6 Ave")]
evening_data_station3=evening_data[(evening_data['start_station_name']=="W 21 St & 6 Ave") | 
                        (durations_df['end_station_name']=="W 21 St & 6 Ave")]

In [ ]:
bike_change = []
for index, row in evening_data_station3.iterrows():
    if row["start_station_name"] == "W 21 St & 6 Ave" and row["end_station_name"] != "W 21 St & 6 Ave":
        bike_change.append(-1)
    elif row["end_station_name"] == "W 21 St & 6 Ave" and row["start_station_name"] != "W 21 St & 6 Ave":
        bike_change.append(1)
    else:
        bike_change.append(0)
evening_data_station3["bike_change"] = bike_change

In [ ]:
morning_station1_blocked = morning_data_station1.groupby(["time_block","day_of_month"])["bike_change"].sum()
evening_station1_blocked = evening_data_station1.groupby(["time_block","day_of_month"])["bike_change"].sum()
morning_station2_blocked = morning_data_station2.groupby(["time_block","day_of_month"])["bike_change"].sum()
evening_station2_blocked = evening_data_station2.groupby(["time_block","day_of_month"])["bike_change"].sum()
morning_station3_blocked = morning_data_station3.groupby(["time_block","day_of_month"])["bike_change"].sum()
evening_station3_blocked = evening_data_station3.groupby(["time_block","day_of_month"])["bike_change"].sum()

In [ ]:
from collections import Counter
#morning_station1_blocked.values is an array of all the bike change rates over all 5 minute blocks for station 1
#during the morning time block
counts = Counter(morning_station1_blocked.values)
rates_m1=pd.DataFrame(columns=["bike_change","probabilities"])
bikes = []
probabilities = []

total=0
for i in counts:
    total += counts[i]
for i in counts:
    bikes.append(i)
    probabilities.append(counts[i] / total)

rates_m1["bike_change"] = bikes
rates_m1["probabilities"] = probabilities
bike_change_m1= rates_m1["bike_change"]
rates_m1 = rates_m1.set_index("bike_change")

In [ ]:
def P_Matrix(N,)
    for i in range(N):
        for j in range(N):
            if j == i:
                P_m1[i][j] = float(rates_m1.loc[0])
            elif j>i and j-i in bike_change_m1.values:
                P_m1[i][j] = float(rates_m1.loc[j-i])
            elif i>j and j-i in bike_change_m1.values:
                P_m1[i][j] = float(rates_m1.loc[j-i])
            else:
                P_m1[i][j] = 0
    return 

In [ ]:
def find_stationary(P):
    from random import seed
    from random import random
    rowlen = P.shape[1]
    state=np.array([np.zeros(rowlen)])
    state[0][0] = 1
    stateHist=state
    dfStateHist=pd.DataFrame(state)
    distr_hist = [np.zeros(rowlen)]
    for x in range(200):
        state=np.dot(state,P)
        stateHist=np.append(stateHist,state,axis=0)
        dfDistrHist = pd.DataFrame(stateHist)

    dfDistrHist.plot(alpha=0.25)
    print(state)
    plt.legend('',frameon=False)
    plt.xlabel("iteration")
    plt.show()
    plt.plot(np.arange(rowlen),np.ravel(state))
    plt.ylabel('long term fraction of time at a state')
    plt.xlabel("# of bikes")
    plt.show()